In [43]:
import os
import tensorflow as tf
import pandas as pd

In [1]:
import numpy as np
import shutil
import matplotlib
import matplotlib.pyplot as plt


In [45]:
pwd

'C:\\Users\\Kevin Tang\\Documents\\HackTheNorth2023\\AI'

In [46]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 1024

In [47]:
DATA_PATH = "dataset/train_data.csv"
TEST_PATH = "dataset/test_data.csv"

In [177]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]])
df.at[0, 2]

3

In [128]:
df = pd.read_csv(DATA_PATH)
print(df)

       BookingID  LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  \
0              1        10         2018             3           31   
1              2       116         2018             2           28   
2              3        11         2018             7           25   
3              4         3         2017             9           12   
4              5        28         2018             3            7   
...          ...       ...          ...           ...          ...   
29015      29016         2         2018            11           21   
29016      29017        18         2018             4           12   
29017      29018         8         2017             9           29   
29018      29019        20         2018             9           18   
29019      29020       274         2018             7           31   

       NumWeekendNights  NumWeekNights      MealPlan  Parking     RoomType  \
0                     0              1   Meal Plan 1        0  Room_Type 1   
1  

In [129]:
df = pd.read_csv(DATA_PATH)

df = df.drop('BookingID', axis=1) # axis: 0 for row, 1 for column

print(len(df))

for i in range(len(df)):
    # ARRIVAL YEAR
    df.at[i, "ArrivalYear"] = df.at[i, "ArrivalYear"] % 2000

    # MEAL PARSING
    if df.at[i, "MealPlan"].startswith("Not"):
        df.at[i, "MealPlan"] = 0
    else:
        df.at[i, "MealPlan"] = df.at[i, "MealPlan"].split()[-1]

    # ROOM TYPE
    df.at[i, "RoomType"] = df.at[i, "RoomType"].split()[-1]

    # MARKET SEGMENT
    if df.at[i, "MarketSegment"] == "Online":
        df.at[i, "MarketSegment"] = 5
    elif df.at[i, "MarketSegment"] == "Offline":
        df.at[i, "MarketSegment"] = 4
    elif df.at[i, "MarketSegment"] == "Corporate":
        df.at[i, "MarketSegment"] = 3
    elif df.at[i, "MarketSegment"] == "Complementary":
        df.at[i, "MarketSegment"] = 2
    elif df.at[i, "MarketSegment"] == "Aviation":
        df.at[i, "MarketSegment"] = 1
    else:
        print("Recheck data.")

    # BOOKING STATUS
    if df.at[i, "BookingStatus"] == "Canceled":
        df.at[i, "BookingStatus"] = 0
    elif df.at[i, "BookingStatus"] == "Not_Canceled":
        df.at[i, "BookingStatus"] = 1

# BOOKING STATUS
target = df.pop("BookingStatus")
df

29020


,LeadTime,ArrivalYear,ArrivalMonth,ArrivalDate,NumWeekendNights,NumWeekNights,MealPlan,Parking,RoomType,NumAdults,NumChildren,MarketSegment,RepeatedGuest,NumPrevCancellations,NumPreviousNonCancelled,AvgRoomPrice,SpecialRequests
0,10,18,3,31,0,1,1,0,1,1,0,3,0,0,0,95.00,0
1,116,18,2,28,2,1,1,0,1,1,0,5,0,0,0,61.00,0
2,11,18,7,25,1,2,1,0,1,2,1,5,0,0,0,129.75,1
3,3,17,9,12,0,1,1,0,1,2,0,5,0,0,0,152.00,3
4,28,18,3,7,1,3,1,0,1,2,0,4,0,0,0,87.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29015,2,18,11,21,1,2,1,0,1,1,0,5,0,0,0,100.67,1
29016,18,18,4,12,0,2,0,0,1,2,0,5,0,0,0,119.00,1
29017,8,17,9,29,0,3,1,0,1,1,0,3,0,0,0,65.00,0
29018,20,18,9,18,2,2,1,0,1,1,0,4,0,0,0,90.00,0


# Normalizing the dataframe

In [218]:
REMOVED_COL = "BookingID"
PREDICT_COL = "BookingStatus"
ENCODING = []

df = pd.read_csv(DATA_PATH)
df = df.drop(REMOVED_COL, axis=1) # axis: 0 for row, 1 for column


for col in df.columns:
    if not all(isinstance(x, (int, float)) for x in df[col]):
        lst = np.unique(df[col].astype(str))
        if col == PREDICT_COL:
            for i in range(len(lst)):
                ENCODING.append((i, lst[i]))
        for i in range(len(lst)):
            df[col] = df[col].replace(lst[i], i)
        lst = []


target = df.pop(PREDICT_COL)
print(target)
print(ENCODING)
df

0        0
1        0
2        1
3        1
4        1
        ..
29015    1
29016    1
29017    1
29018    0
29019    1
Name: BookingStatus, Length: 29020, dtype: int64
[(0, 'Canceled'), (1, 'Not_Canceled')]


,LeadTime,ArrivalYear,ArrivalMonth,ArrivalDate,NumWeekendNights,NumWeekNights,MealPlan,Parking,RoomType,NumAdults,NumChildren,MarketSegment,RepeatedGuest,NumPrevCancellations,NumPreviousNonCancelled,AvgRoomPrice,SpecialRequests
0,10,2018,3,31,0,1,0,0,0,1,0,2,0,0,0,95.00,0
1,116,2018,2,28,2,1,0,0,0,1,0,4,0,0,0,61.00,0
2,11,2018,7,25,1,2,0,0,0,2,1,4,0,0,0,129.75,1
3,3,2017,9,12,0,1,0,0,0,2,0,4,0,0,0,152.00,3
4,28,2018,3,7,1,3,0,0,0,2,0,3,0,0,0,87.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29015,2,2018,11,21,1,2,0,0,0,1,0,4,0,0,0,100.67,1
29016,18,2018,4,12,0,2,3,0,0,2,0,4,0,0,0,119.00,1
29017,8,2017,9,29,0,3,0,0,0,1,0,2,0,0,0,65.00,0
29018,20,2018,9,18,2,2,0,0,0,1,0,3,0,0,0,90.00,0


In [219]:
# Name: BookingStatus, Length: 29020, dtype: object
# dtype can't be object or will get error for normalizer(numeric_features.iloc[:3])
target = target.astype('int')
df = df.astype('int')
print(type(target))
target

<class 'pandas.core.series.Series'>


0        0
1        0
2        1
3        1
4        1
        ..
29015    1
29016    1
29017    1
29018    0
29019    1
Name: BookingStatus, Length: 29020, dtype: int32

In [232]:
numeric_feature_names = [name for name in df.columns]
# numeric_feature_names = ['LeadTime', 'ArrivalYear', 'ArrivalMonth',  'ArrivalDate', 'NumWeekendNights', "NumWeekNights", "MealPlan", "Parking", "RoomType", "NumAdults", "NumChildren", "MarketSegment", "RepeatedGuest", "NumPrevCancellations", "NumPreviousNonCancelled", "AvgRoomPrice", "SpecialRequests"]
numeric_features = df[numeric_feature_names]
numeric_features.head()

print(type(numeric_features))

<class 'pandas.core.frame.DataFrame'>


In [233]:
# numeric_features = np.asarray(numeric_features).astype('float32')

tf.convert_to_tensor(numeric_features)


<tf.Tensor: shape=(29020, 17), dtype=int32, numpy=
array([[  10, 2018,    3, ...,    0,   95,    0],
       [ 116, 2018,    2, ...,    0,   61,    0],
       [  11, 2018,    7, ...,    0,  129,    1],
       ...,
       [   8, 2017,    9, ...,    0,   65,    0],
       [  20, 2018,    9, ...,    0,   90,    0],
       [ 274, 2018,    7, ...,    0,   96,    1]])>

In [234]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features.to_numpy())


In [235]:
numeric_features = pd.DataFrame(numeric_features)
type(numeric_features)


pandas.core.frame.DataFrame

In [236]:
normalizer(numeric_features.iloc[:3])


<tf.Tensor: shape=(3, 17), dtype=float32, numpy=
array([[-0.87647665,  0.46875393, -1.4414891 ,  1.7625139 , -0.9305437 ,
        -0.84950244, -0.49299878, -0.17818205, -0.50807035, -1.6334727 ,
        -0.2592433 , -2.279335  , -0.16221005, -0.06437863, -0.08737914,
        -0.23430355, -0.7885584 ],
       [ 0.3577327 ,  0.46875393, -1.7678146 ,  1.4194049 ,  1.3596122 ,
        -0.84950244, -0.49299878, -0.17818205, -0.50807035, -1.6334727 ,
        -0.2592433 ,  0.6553474 , -0.16221005, -0.06437863, -0.08737914,
        -1.2049241 , -0.7885584 ],
       [-0.8648332 ,  0.46875393, -0.13618678,  1.076296  ,  0.2145343 ,
        -0.14614795, -0.49299878, -0.17818205, -0.50807035,  0.29529724,
         2.2228546 ,  0.6553474 , -0.16221005, -0.06437863, -0.08737914,
         0.73631704,  0.482632  ]], dtype=float32)>

In [237]:
print(normalizer)

In [238]:
def get_basic_model():
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(len(ENCODING), activation='sigmoid')
        ])

    model.compile(optimizer='adam',
                    loss="sparse_categorical_crossentropy", # tf.keras.losses.BinaryCrossentropy(from_logits=True)
                    metrics=['accuracy'])
    return model

get_basic_model().summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_12 (Normaliz  (None, 17)                35        
 ation)                                                          
                                                                 
 dense_65 (Dense)            (None, 128)               2304      
                                                                 
 dense_66 (Dense)            (None, 64)                8256      
                                                                 
 dropout_39 (Dropout)        (None, 64)                0         
                                                                 
 dense_67 (Dense)            (None, 32)                2080      
                                                                 
 dropout_40 (Dropout)        (None, 32)                0         
                                                     

In [239]:
print(numeric_features.iloc[:3])

   LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  NumWeekendNights  \
0        10         2018             3           31                 0   
1       116         2018             2           28                 2   
2        11         2018             7           25                 1   

   NumWeekNights  MealPlan  Parking  RoomType  NumAdults  NumChildren  \
0              1         0        0         0          1            0   
1              1         0        0         0          1            0   
2              2         0        0         0          2            1   

   MarketSegment  RepeatedGuest  NumPrevCancellations  \
0              2              0                     0   
1              4              0                     0   
2              4              0                     0   

   NumPreviousNonCancelled  AvgRoomPrice  SpecialRequests  
0                        0            95                0  
1                        0            61                0  
2    

In [240]:
model = get_basic_model()
history = model.fit(numeric_features.to_numpy(), target, epochs=30, batch_size=BATCH_SIZE)


Epoch 1/30
29/29 [==============================] - 7s 166ms/step - loss: 0.5996 - accuracy: 0.6677
Epoch 2/30
29/29 [==============================] - 4s 157ms/step - loss: 0.4796 - accuracy: 0.7780
Epoch 3/30
29/29 [==============================] - 6s 190ms/step - loss: 0.4474 - accuracy: 0.7999
Epoch 4/30
29/29 [==============================] - 5s 153ms/step - loss: 0.4255 - accuracy: 0.8099
Epoch 5/30
29/29 [==============================] - 5s 175ms/step - loss: 0.4123 - accuracy: 0.8168
Epoch 6/30
29/29 [==============================] - 5s 184ms/step - loss: 0.4009 - accuracy: 0.8229
Epoch 7/30
29/29 [==============================] - 5s 168ms/step - loss: 0.3916 - accuracy: 0.8281
Epoch 8/30
29/29 [==============================] - 5s 176ms/step - loss: 0.3826 - accuracy: 0.8329
Epoch 9/30
29/29 [==============================] - 5s 172ms/step - loss: 0.3739 - accuracy: 0.8356
Epoch 10/30
29/29 [==============================] - 5s 181ms/step - loss: 0.3670 - accuracy: 0.8408

In [241]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(3):
  print(row)


(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([  10, 2018,    3,   31,    0,    1,    0,    0,    0,    1,    0,
          2,    0,    0,    0,   95,    0])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([ 116, 2018,    2,   28,    2,    1,    0,    0,    0,    1,    0,
          4,    0,    0,    0,   61,    0])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(17,), dtype=int32, numpy=
array([  11, 2018,    7,   25,    1,    2,    0,    0,    0,    2,    1,
          4,    0,    0,    0,  129,    1])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [242]:
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()
history = model.fit(numeric_batches, epochs=90)

Epoch 1/90
29/29 [==============================] - 1s 3ms/step - loss: 0.5884 - accuracy: 0.6646
Epoch 2/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4810 - accuracy: 0.7797
Epoch 3/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4492 - accuracy: 0.7984
Epoch 4/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4284 - accuracy: 0.8129
Epoch 5/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4141 - accuracy: 0.8184
Epoch 6/90
29/29 [==============================] - 0s 3ms/step - loss: 0.4054 - accuracy: 0.8275
Epoch 7/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3976 - accuracy: 0.8300
Epoch 8/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3862 - accuracy: 0.8337
Epoch 9/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3791 - accuracy: 0.8374
Epoch 10/90
29/29 [==============================] - 0s 3ms/step - loss: 0.3716 - accuracy: 0.8421
Epoch 11/90
29/29 [

In [243]:
plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

# 💀💀💀💀💀💀💀💀💀

NameError: name 'plt' is not defined

In [244]:
# pd.DataFrame(numeric_features.numpy())
print(type(numeric_features))
print(numeric_features)
predictions = model.predict(np.array(numeric_features.iloc[:20]))

print("asdf")

lst = ["Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled",
"Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Not_Canceled",
"Canceled"]
# print(predictions)

for i in range(len(predictions)-1):
    print(f"{predictions[i]}: {lst[i]}")
    if predictions[i][0] > predictions[i][1]:
        print("Predicted: cancled")
    else:
        print("Predicted: not_canceled")


<class 'pandas.core.frame.DataFrame'>
       LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  NumWeekendNights  \
0            10         2018             3           31                 0   
1           116         2018             2           28                 2   
2            11         2018             7           25                 1   
3             3         2017             9           12                 0   
4            28         2018             3            7                 1   
...         ...          ...           ...          ...               ...   
29015         2         2018            11           21                 1   
29016        18         2018             4           12                 0   
29017         8         2017             9           29                 0   
29018        20         2018             9           18                 2   
29019       274         2018             7           31                 2   

       NumWeekNights  MealPlan  Parki